In [1]:
import psycopg2

In [4]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user='postgres', password='Root123$', host='127.0.0.1', port= '5432'
)
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()


In [12]:
sql_dim_film = '''
CREATE TABLE IF NOT EXISTS olap_schema.dim_film AS 
SELECT DISTINCT film_id,
       title, 
       release_year, 
       length as duration,
       last_update
FROM oltp_schema.film
'''

sql_dim_customer = ''' 
CREATE TABLE IF NOT EXISTS olap_schema.dim_customer AS 
SELECT customer_id, first_name ||' '|| last_name AS customer_name,
       email, create_date, last_update
FROM oltp_schema.customer
'''


sql_dim_staff = ''' 
CREATE TABLE IF NOT EXISTS olap_schema.dim_staff AS 
SELECT DISTINCT staff_id, first_name ||' '|| last_name AS staff_name,
       email, last_update
FROM oltp_schema.staff
'''

sql_dim_date = '''
CREATE TABLE IF NOT EXISTS olap_schema.dim_date AS 
SELECT TO_CHAR(ax.payment_date, 'YYYYMMDD') date_id,
        CAST(TO_CHAR(ax.payment_date, 'YYYY-MM-DD') AS DATE) date_,
       TO_CHAR(ax.payment_date, 'YYYY') year_,
       TO_CHAR(ax.payment_date, 'Q') quarter_,
       TO_CHAR(ax.payment_date, 'MON') month_,
       TO_CHAR(ax.payment_date, 'DD') day_of_week,
       TO_CHAR(ax.payment_date, 'IW') week_of_year
FROM oltp_schema.payment AX
'''

sql_fact_transaction = '''
CREATE TABLE if not exists olap_schema.fact_transacion AS
SELECT DISTINCT bx.customer_id,
       cx.staff_id, dx.rental_id,
       TO_CHAR(ax.payment_date, 'YYYYMMDD') date_id,
       SUM(ax.amount) total_revenue,
       AVG(ax.amount) avg_revenue
FROM oltp_schema.payment AX
LEFT JOIN oltp_schema.customer BX ON bx.customer_id = ax.customer_id
LEFT JOIN oltp_schema.staff CX ON cx.staff_id = ax.customer_id
LEFT JOIN oltp_schema.rental DX ON dx.rental_id = ax.customer_id
GROUP BY ax.payment_id, bx.customer_id,
         cx.staff_id, dx.rental_id,
         TO_CHAR(ax.payment_date, 'YYYYMMDD')
'''

cursor.execute(sql_dim_film)
cursor.execute(sql_dim_customer)
cursor.execute(sql_dim_staff)
cursor.execute(sql_dim_date)
cursor.execute(sql_fact_transaction)

print("Tables created successfully........")

conn.commit()


Tables created successfully........
